In [1]:
import json
import pymongo
import tweepy

with open('/credentials/consumer_key.txt', 'r') as f:
    consumer_key =  f.read().strip()
f.closed

with open('/credentials/consumer_secret.txt', 'r') as f:
    consumer_secret = f.read().strip()
f.closed

with open('/credentials/access_key.txt', 'r') as f:
    access_key = f.read().strip()
f.closed

with open('/credentials/access_secret.txt', 'r') as f:
     access_secret = f.read().strip()
f.closed

#Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)


FileNotFoundError: [Errno 2] No such file or directory: 'consumer_key.txt'

Using a Standard API you can only access tweets from the last 7 days, so there is no need for a filter. However, we should store those tweets if we want to create historical data, otherwise we won't have access to them after 7 days. We can also restrict the language of the tweets, which is a good idea since our machine learning model will only work on English tweets. We can restrict the geolocation using geo_search or using coordinates inside api.search within the parameter geocode. The tweets are chosen randomly. However there are some limitiations. Only 180 queries are allowed every 15min. In 180 queries we can get approximately seven thousand tweets. 

In [ ]:
places = api.geo_search(query="New York", granularity="city")
place_id = places[0].id

tweets = api.search(q="place:%s" % place_id, lang='en', count=10)
for tweet in tweets:
    print(tweet.text + " | " + tweet.place.name if tweet.place else "Undefined place")

If we want to get a high amount of tweets we can loop over the maximum number of queries. The following cell keeps storing tweets to the mongo db database and freezes 15 minutes every time we reach the maximum number of queries. We can break the loop whenever we feel we have enough tweets by changing the parameter K.

In [ ]:
import time

max_queries = 180
count = 0
K = 50

while True:
    for i in range(max_queries):
        tweets = api.search(q="place:%s" % place_id, lang='en', count=1000)
            for tweet in tweets:
                #save to mongodb
                pass
    count +=1
    if count > K: break
    time.sleep(910)

We can also stream tweets from New York

In [3]:
from tweepy import Stream,StreamListener

class listener(StreamListener):
    def __init__(self):
        super(StreamListener, self).__init__()
        self.num_tweets = 0
        
    def on_data(self, status):
        if self.num_tweets < 5:
            json_data=json.loads(status)
            print (str('** '+json_data["user"]["screen_name"])+' ** : ' + json_data["text"])
            self.num_tweets += 1
            return True
        else:
            return False
        
    def on_error(self, status):
        print (status)
        
# Catch all tweets in New York area and print them
twitterStream = Stream(auth, listener()) 
twitterStream.filter(locations=[-74.1781,40.5609,-73.7468,40.8866])
print ("done")

** esjacobs ** : @WalshFreedom you yesterday: “vote for republicans!”
** Silvanita2812 ** : “Todos me dicen la negra, llorona...” https://t.co/FMJJNpT7xz
** Brittnaeee ** : The city is mine📍 https://t.co/JxknCDfAfb
** Aoiferocksitout ** : @brooklynburning Saaaaame!! ❤️❤️
** spanishzac ** : PEOPLE OVER 75 SHOULD NOT BE ALLOWED TO VOTE!!!!! They will not be here to see the consequences of their actions!!!!
done
